In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.5
    Uninstalling langchain-text-splitters-0.3.5:
      Successfully uninstalled langchain-text-splitters-0.3.5
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.17
    Uninstalling langchain-0.3.17:
      Successfully uninstalled langchain-0.3.17


In [ ]:
# from langchain.agents import create_sql_agent
# from langchain.chat_models import ChatOpenAI
# from langchain.sql_database import SQLDatabase
# import sqlite3

from langchain.agents import create_sql_agent
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
import sqlite3

In [ ]:
# Step 1: Set up a sample SQLite database
def create_sample_database():
    connection = sqlite3.connect("sample.db")
    cursor = connection.cursor()

    # Create tables
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS startups (
        id INTEGER PRIMARY KEY,
        name TEXT,
        domain TEXT,
        founding_year INTEGER
    )
    """)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS founders (
        id INTEGER PRIMARY KEY,
        name TEXT,
        startup_id INTEGER,
        FOREIGN KEY (startup_id) REFERENCES startups (id)
    )
    """)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS founders_details (
        id INTEGER PRIMARY KEY,
        full_name TEXT,
        location TEXT,
        contact TEXT,
        email_id TEXT,
        founders_id INTEGER,
        FOREIGN KEY (founders_id) REFERENCES founders (id)
    )
    """)

    # Insert sample data
    cursor.execute("INSERT INTO startups (name, domain, founding_year) VALUES ('HealthAI', 'healthcare', 2021)")
    cursor.execute("INSERT INTO startups (name, domain, founding_year) VALUES ('MediTech', 'healthcare', 2019)")
    cursor.execute("INSERT INTO startups (name, domain, founding_year) VALUES ('AIWorks', 'finance', 2022)")
    cursor.execute("INSERT INTO startups (name, domain, founding_year) VALUES ('AVHealthCareAI', 'healthcare', 2024)")

    cursor.execute("INSERT INTO founders (name, startup_id) VALUES ('Alice', 1)")
    cursor.execute("INSERT INTO founders (name, startup_id) VALUES ('Bob', 2)")
    cursor.execute("INSERT INTO founders (name, startup_id) VALUES ('Charlie', 3)")
    cursor.execute("INSERT INTO founders (name, startup_id) VALUES ('Anand', 4)")

    cursor.execute("INSERT INTO founders_details (full_name, location, contact, email_id, founders_id) VALUES ('Alice Johnson','Kenya', '+254 2393234','alice@gmail.com', 1)")
    cursor.execute("INSERT INTO founders_details (full_name, location, contact, email_id, founders_id) VALUES ('Bob Marly','Bandra', '+91 709239234','bob@gmail.com', 2)")
    cursor.execute("INSERT INTO founders_details (full_name, location, contact, email_id, founders_id) VALUES ('Charlie Champs','Andheri', '+91 77123428644','charlie@gmail.com', 3)")
    cursor.execute("INSERT INTO founders_details (full_name, location, contact, email_id, founders_id) VALUES ('Anand Vishwakarma','Thane', '+91 88338822944','anand.j.vishwakarma@gmail.com', 4)")

    connection.commit()
    connection.close()

In [ ]:
# Step 2: Initialize the LangChain SQL agent
def initialize_agent():
    # Connect to the SQLite database
    db = SQLDatabase.from_uri("sqlite:///sample.db")
    llm = ChatOpenAI(model="gpt-4", temperature=0)
    toolkit = SQLDatabaseToolkit(db=db, llm=llm)
    agent = create_sql_agent(
        llm=llm,
        toolkit=toolkit,
        verbose=True
    )
    return agent

In [ ]:
# Step 3: Query the agent
def query_agent(agent, query):
    response = agent.run(query)
    print("\n### Response ###")
    print(response)

In [ ]:
create_sample_database()

In [ ]:
from google.colab import userdata
my_secret_key = userdata.get('OPENAI_API_KEY')

# Set OpenAI API key
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
# Initialize the multi-hop agent
agent = initialize_agent()

<ipython-input-4-6be8a4577b95>:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4", temperature=0)


In [ ]:
# Example query: Multi-hop reasoning
query = """
Which startups founded after 2020 are working in the healthcare domain,
and who are their founders? Also, can you provide their contact details, emails and their region.
"""

query_agent(agent, query)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""founders, founders_details, startupsThe tables "founders", "founders_details", and "startups" seem relevant to the question. I should check their schemas to understand their structure and the data they contain.
Action: sql_db_schema
Action Input: "founders, founders_details, startups"
CREATE TABLE founders (
	id INTEGER, 
	name TEXT, 
	startup_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(startup_id) REFERENCES startups (id)
)

/*
3 rows from founders table:
id	name	startup_id
1	Alice	1
2	Bob	2
3	Charlie	3
*/


CREATE TABLE founders_details (
	id INTEGER, 
	full_name TEXT, 
	location TEXT, 
	contact TEXT, 
	email_id TEXT, 
	founders_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(founders_id) REFERENCES founders (id)
)

/*
3 rows from founders_details table:
id	full_name	location	contact	email_id	founders_id
1	Alice Johnson	Kenya	+254 2393234	alice@gmail.com	1
2	Bob Marly	Bandra	+91 709239234	bob@gmai